In [1]:
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

dataset_path = os.listdir('/content/Theft detection')

label_types = os.listdir('/content/Theft detection')
print (label_types)

['.ipynb_checkpoints', 'normal', 'shoplifting']


In [11]:
rooms = []

for item in dataset_path:
 # Get all the file names
 all_rooms = os.listdir('/content/Theft detection' + '/' +item)

 # Add them to the list
 for room in all_rooms:
    rooms.append((item, str('/content/Theft detection' + '/' +item) + '/' + room))

# Build a dataframe
train_df = pd.DataFrame(data=rooms, columns=['tag', 'video_name'])
print(train_df.head())
print(train_df.tail())

      tag                                       video_name
0  normal  /content/Theft detection/normal/Normal (40).mp4
1  normal  /content/Theft detection/normal/Normal (17).mp4
2  normal  /content/Theft detection/normal/Normal (61).mp4
3  normal  /content/Theft detection/normal/Normal (65).mp4
4  normal  /content/Theft detection/normal/Normal (51).mp4
             tag                                         video_name
177  shoplifting  /content/Theft detection/shoplifting/Shoplifti...
178  shoplifting  /content/Theft detection/shoplifting/Shoplifti...
179  shoplifting  /content/Theft detection/shoplifting/Shoplifti...
180  shoplifting  /content/Theft detection/shoplifting/Shoplifti...
181  shoplifting  /content/Theft detection/shoplifting/Shoplifti...


In [12]:
rooms = []

for item in dataset_path:
 # Get all the file names
 all_rooms = os.listdir('/content/Theft detection' + '/' +item)

 # Add them to the list
 for room in all_rooms:
    rooms.append((item, str('/content/Theft detection' + '/' +item) + '/' + room))

# Build a dataframe
train_df = pd.DataFrame(data=rooms, columns=['tag', 'video_name'])
print(train_df.head())
print(train_df.tail())

In [13]:
train_df = pd.read_csv("train.csv")


print(f"Total videos for training: {len(train_df)}")


train_df.sample(10)

Total videos for training: 182


,Unnamed: 0,video_name,tag
143,143,/content/Theft detection/shoplifting/Shoplifti...,shoplifting
154,154,/content/Theft detection/shoplifting/Shoplifti...,shoplifting
65,65,/content/Theft detection/normal/Normal (19).mp4,normal
134,134,/content/Theft detection/shoplifting/Shoplifti...,shoplifting
59,59,/content/Theft detection/normal/Normal (77).mp4,normal
83,83,/content/Theft detection/normal/Normal (59).mp4,normal
165,165,/content/Theft detection/shoplifting/Shoplifti...,shoplifting
44,44,/content/Theft detection/normal/Normal (30).mp4,normal
76,76,/content/Theft detection/normal/Normal (57).mp4,normal
56,56,/content/Theft detection/normal/Normal (24).mp4,normal


In [15]:
import os
import cv2
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# Define dataset path containing folder names "normal" and "shoplifting"
dataset_path = ["normal", "shoplifting"]
root_dir = '/content/Theft detection'

# Function to extract frames from videos
def extract_frames(video_path, frame_skip=30, target_size=(224, 224)):
    frames = []
    cap = cv2.VideoCapture(video_path)
    frame_count = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frame_count += 1
        if frame_count % frame_skip == 0:
            frame = cv2.resize(frame, target_size)
            frames.append(frame)
    cap.release()
    return frames

# Function to preprocess videos and extract features
def preprocess_videos(root_dir, dataset_path, frame_skip=30, target_size=(224, 224)):
    frames_data = []
    for class_folder in dataset_path:
        class_path = os.path.join(root_dir, class_folder)
        for video_file in os.listdir(class_path):
            video_path = os.path.join(class_path, video_file)
            frames = extract_frames(video_path, frame_skip, target_size)
            frames_data.extend([(class_folder, video_file, frame) for frame in frames])
    return frames_data

# Preprocess videos and extract features
frames_data = preprocess_videos(root_dir, dataset_path)

# Create DataFrame
train_df = pd.DataFrame(frames_data, columns=['tag', 'video_name', 'frame'])

# Encode labels
label_encoder = LabelEncoder()
train_df['tag_encoded'] = label_encoder.fit_transform(train_df['tag'])

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(train_df['frame'], train_df['tag_encoded'], test_size=0.2, random_state=42)

# Convert frames to numpy arrays
X_train = np.array(X_train.tolist())
X_test = np.array(X_test.tolist())

# Normalize pixel values
X_train = X_train / 255.0
X_test = X_test / 255.0

# Build CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.1)

# Evaluate model
y_pred = (model.predict(X_test) > 0.5).astype("int32")
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)

# Print evaluation metrics
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("Confusion Matrix:\n", conf_matrix)

# Save report to a file
report = f"""\
--- Theft Detection Algorithm Report ---

Data Preprocessing:
- Dataset path: {root_dir}
- Frame skip: 30 frames
- Target frame size: (224, 224)

Model Building:
- CNN model architecture:
  - Input Layer: Conv2D with 32 filters and relu activation
  - MaxPooling Layer: MaxPooling2D with (2, 2) pool size
  - Conv2D Layer: Conv2D with 64 filters and relu activation
  - MaxPooling Layer: MaxPooling2D with (2, 2) pool size
  - Flatten Layer
  - Dense Layer: 64 units with relu activation
  - Output Layer: Dense Layer with sigmoid activation

Evaluation Results:
- Accuracy: {accuracy}
- Precision: {precision}
- Recall: {recall}
- F1 Score: {f1}
- Confusion Matrix:
{conf_matrix}

Discussion:
- The CNN model achieved good accuracy, precision, recall, and F1 score in detecting shoplifting activities.

"""

with open("theft_detection_report.txt", "w") as file:
    file.write(report)


Epoch 1/10
43/43 [==============================] - 116s 3s/step - loss: 0.6841 - accuracy: 0.7287 - val_loss: 0.4503 - val_accuracy: 0.8039
Epoch 2/10
43/43 [==============================] - 118s 3s/step - loss: 0.2411 - accuracy: 0.9040 - val_loss: 0.3601 - val_accuracy: 0.8366
Epoch 3/10
43/43 [==============================] - 115s 3s/step - loss: 0.1589 - accuracy: 0.9389 - val_loss: 0.2693 - val_accuracy: 0.8758
Epoch 4/10
43/43 [==============================] - 116s 3s/step - loss: 0.1142 - accuracy: 0.9571 - val_loss: 0.2631 - val_accuracy: 0.8627
Epoch 5/10
43/43 [==============================] - 115s 3s/step - loss: 0.0909 - accuracy: 0.9636 - val_loss: 0.2902 - val_accuracy: 0.8627
Epoch 6/10
43/43 [==============================] - 116s 3s/step - loss: 0.0861 - accuracy: 0.9702 - val_loss: 0.2778 - val_accuracy: 0.8758
Epoch 7/10
43/43 [==============================] - 117s 3s/step - loss: 0.0714 - accuracy: 0.9767 - val_loss: 0.3112 - val_accuracy: 0.8824
Epoch 8/10
43